In [1]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
import csv, os, datetime

image_dir = "images/"
colpali_output = "colpali_output.csv"
vlm_log = "vlm_log.csv"
model_name = "qwen_2_VL_2B_Instruct"

In [2]:
# Get total GPU memory
total_memory = torch.cuda.get_device_properties(0).total_memory

# Get free and total memory
free_memory, total_memory = torch.cuda.mem_get_info()

# Get allocated and reserved memory
allocated_memory = torch.cuda.memory_allocated()
reserved_memory = torch.cuda.memory_reserved()

# Print memory information
print(f"Total memory: {total_memory / (1024**3):.2f} GB")
print(f"Free memory: {free_memory / (1024**3):.2f} GB")
print(f"Allocated memory: {allocated_memory / (1024**3):.2f} GB")
print(f"Reserved memory: {reserved_memory / (1024**3):.2f} GB")

Total memory: 15.70 GB
Free memory: 11.91 GB
Allocated memory: 0.00 GB
Reserved memory: 0.00 GB


In [1]:
def get_model_processor():
    model = Qwen2VLForConditionalGeneration.from_pretrained(
        "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto"
    )
    processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")
    return model, processor
    
def generate_response(image, prompt, model, processor):
    messages = [
        {
            "role": "system",
            "content": "If you do not know the answer, do not answer."
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": image_dir + image,
                },
                {"type": "text", "text": prompt},
            ],
        }
    ]

    text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    generated_ids = model.generate(**inputs, max_new_tokens=128)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )

    return output_text

def log_response(doc_id, document, page_num, question, response):

    isfile = os.path.isfile(vlm_log)
    with open(vlm_log, mode='a', newline='') as file:
        writer = csv.writer(file)
    
        if not isfile:
            writer.writerow(["doc_id", "document", "page_num", "question", "response", "model", "datetime"])
        
        writer.writerow([doc_id, document, page_num, question, response, model_name, str(datetime.datetime.now())])

In [6]:
model, processor = get_model_processor()

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
with open(colpali_output, mode='r', newline='') as file:
    reader = csv.reader(file)
    
    header = next(reader)

    for row_num, row in enumerate(reader, start=1):
        doc_id, document, page_num, score, question, base64 = row
        response = generate_response(base64, question, model, processor)
        log_response(doc_id, document, page_num, question, response[0])